In [31]:
import os
import pandas as pd

In [2]:
?pd.read_csv

In [77]:
df = pd.read_csv("Kallisto_names.txt", header=0, sep="\t")
df.head()

,table,index,run_accession,Renamed
0,B73v4all,B73v4,SRR5909740,Root
1,B73v4all,B73v4,SRR5909741,Root
2,B73v4all,B73v4,SRR5909742,Root
3,B73v4all,B73v4,SRR5909746,Root
4,B73v4all,B73v4,SRR5909845,Root


In [80]:
uniq_genotypes = set(list(df["table"]))

In [83]:
folder_list = []
for row in df[df["index"]=="Oh7B"].iloc[-2:].iterrows():
    row = list(row[1])
    table, index, treatment, file_name = row[0], row[1], row[3], row[2]
    folder_list.append(str(table+"_"+index+"_"+treatment))
    os.system("""kallisto quant --single -l 90 -s 1 -t 32 --plaintext \
                 -i Kallisto/{0}.idx -o kallisto_results/{1}_{2}_{3}/ {4}.fp.fastq.gz """.format(index, table, index, treatment, file_name))


[quant] fragment length distribution is truncated gaussian with mean = 90, sd = 1
[index] k-mer length: 31
[index] number of targets: 40,334
[index] number of k-mers: 40,519,216
[index] number of equivalence classes: 122,118
[quant] running in single-end mode
[quant] will process file 1: SRR5910663.fp.fastq.gz
[quant] finding pseudoalignments for the reads ... done
[quant] processed 10,538,328 reads, 7,031,674 reads pseudoaligned
[   em] quantifying the abundances ... done
[   em] the Expectation-Maximization algorithm ran for 846 rounds


[quant] fragment length distribution is truncated gaussian with mean = 90, sd = 1
[index] k-mer length: 31
[index] number of targets: 40,334
[index] number of k-mers: 40,519,216
[index] number of equivalence classes: 122,118
[quant] running in single-end mode
[quant] will process file 1: SRR5910668.fp.fastq.gz
[quant] finding pseudoalignments for the reads ... done
[quant] processed 4,747,301 reads, 3,214,671 reads pseudoaligned
[   em] quantifying 

In [85]:
first = True
col_list = []
for fl in folder_list:
    fls = fl.split("_")
    if fls[0] == fls[1]:
        col_list.append(fls[2])
    else:
        col_list.append(fls[1]+"_"+fls[2])
    if first:
        tpm = pd.read_csv("kallisto_results/"+fl+"/abundance.tsv", sep="\t", header=0, index_col=0)["tpm"]
        first = False
    else:
        tpm = pd.merge(tpm, 
                   pd.read_csv("kallisto_results/"+fl+"/abundance.tsv", sep="\t", header=0, index_col=0)["tpm"], 
                   left_index=True, right_index=True)
tpm.columns = col_list
tpm.to_csv("TPM_tables/"+fls[0]+".tpm.tsv", sep="\t")